In [ ]:
import dautil as dl
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import HTML

In [ ]:
def error(data, fit):
    return data - fit


def win_rng():
    return range(3, 25, 2)


def calc_mape(i, j, pres):
    return dl.stats.mape(pres, savgol_filter(pres, i, j))

In [ ]:
pres = dl.data.Weather.load()['PRESSURE'].dropna()
pres = pres.resample('A')

In [ ]:
context = dl.nb.Context('eval_smooth')
lr = dl.nb.LatexRenderer(chapter=6, start=6, context=context)
lr.render(r'Y_j= \sum _{i=-(m-1)/2}^{i=(m-1)/2}C_i\, y_{j+i}\qquad  \frac{m+1}{2} \le j \le n-\frac{m-1}{2}')

In [ ]:
%matplotlib inline
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
cp = dl.plotting.CyclePlotter(sp.ax)
cp.plot(pres.index, pres, label='Pressure')
cp.plot(pres.index, savgol_filter(pres, 11, 2), label='Poly order 2')
cp.plot(pres.index, savgol_filter(pres, 11, 3), label='Poly order 3')
cp.plot(pres.index, savgol_filter(pres, 11, 4), label='Poly order 4')
sp.label(ylabel_params=dl.data.Weather.get_header('PRESSURE'))

cp = dl.plotting.CyclePlotter(sp.next_ax())
stds = [error(pres, savgol_filter(pres, i, 2)).std()
        for i in win_rng()]
cp.plot(win_rng(), stds, label='Filtered')
stds = [error(pres, pd.rolling_mean(pres, i)).std()
        for i in win_rng()]
cp.plot(win_rng(), stds, label='Rolling mean')
sp.label()

sp.label(advance=True)
sp.ax.boxplot([error(pres, savgol_filter(pres, i, 2))
            for i in win_rng()])
sp.ax.set_xticklabels(win_rng())

sp.label(advance=True)
df = dl.report.map_grid(win_rng()[1:], range(1, 5),
                 ['win_size', 'poly', 'mape'], calc_mape, pres)
sns.heatmap(df, cmap='Blues', ax=sp.ax)
HTML(sp.exit())